In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score, explained_variance_score

from sklearn.ensemble import RandomForestRegressor 


In [2]:

df = pd.read_csv('/Users/snesbitt/python/ATMS 523/Module 5/homework/radar_parameters.csv')
df.drop(columns='Unnamed: 0', inplace=True)
df['R M-P'] = 


In [3]:
df.columns

Index(['Zh (dBZ)', 'Zdr (dB)', 'Ldr (dB)', 'Kdp (deg km-1)', 'Ah (dBZ/km)',
       'Adr (dB/km)', 'R (mm/hr)'],
      dtype='object')

In [4]:
selected_columns = ['Zh (dBZ)', 'Zdr (dB)', 'Ldr (dB)', 'Kdp (deg km-1)', 'Ah (dBZ/km)',
       'Adr (dB/km)', 'R (mm/hr)']

In [5]:
df_processed = df[selected_columns].reset_index(drop=True)
target_label = 'R (mm/hr)'
input_feat = list(set(selected_columns).difference(set([target_label])))

In [6]:
df_X = df_processed[input_feat].reset_index(drop=True)
df_y = df_processed[target_label]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.3, random_state=42)

In [9]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
import numpy as np

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(StandardScaler(), PolynomialFeatures(degree), LinearRegression(**kwargs))

param_grid = {'polynomialfeatures__degree': np.arange(8), 'linearregression__fit_intercept': [True]}

poly_grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

poly_grid.fit(X_train, y_train)

Fitting 7 folds for each of 9 candidates, totalling 63 fits
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=0; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=0; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=0; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=0; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=0; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=0; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=0; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=1; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=1; total time=   0.0s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=1;

/Users/snesbitt/miniforge3/envs/py3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=6; total time=   2.4s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=6; total time=   2.5s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=6; total time=   2.5s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=7; total time=  11.1s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=7; total time=  11.2s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=7; total time=  11.3s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=7; total time=  11.3s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=7; total time=  10.7s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=7; total time=  11.6s
[CV] END linearregression__fit_intercept=True, polynomialfeatures__degree=7; total time=  11.8s
[CV] END linearregression__fit_intercept

GridSearchCV(cv=7,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             n_jobs=-1,
             param_grid={'linearregression__fit_intercept': [True],
                         'polynomialfeatures__degree': array([0, 1, 2, 3, 4, 5, 6, 7, 8])},
             scoring='neg_mean_squared_error', verbose=2)

In [7]:
poly_grid = GridSearchCV(PolynomialRegression(), param_grid, 
                         cv=10, 
                         scoring='neg_mean_squared_error', 
                         verbose=3) 

NameError: name 'PolynomialRegression' is not defined

In [12]:
rf_params= {'bootstrap': [True, False],  
   'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  
   'max_features': [1, 'sqrt'],  
   'min_samples_leaf': [1, 2, 4],  
   'min_samples_split': [2, 5, 10],  
   'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [13]:
# Create base LGBM model
rf_base = RandomForestRegressor(random_state=42)

In [15]:
# Create grid search for LGBM model
rf_grid = RandomizedSearchCV(estimator=rf_base, param_distributions=rf_params, 
                                n_iter=100, verbose=2, cv=5,
                                n_jobs=-1)

In [16]:
%%time
# Fit the random search LGBM model
rf_grid.fit(X_train, y_train)

import joblib
filename = 'finalized_model_rf.sav'
joblib.dump(rf_grid, filename)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END bootstrap=True, max_depth=50, max_features=1, min_samples_leaf=1, min_samples_split=10, n_estimators=1200; total time=  10.7s
[CV] END bootstrap=True, max_depth=50, max_features=1, min_samples_leaf=1, min_samples_split=10, n_estimators=1200; total time=  11.0s
[CV] END bootstrap=True, max_depth=50, max_features=1, min_samples_leaf=1, min_samples_split=10, n_estimators=1200; total time=  11.4s
[CV] END bootstrap=True, max_depth=50, max_features=1, min_samples_leaf=1, min_samples_split=10, n_estimators=1200; total time=  11.4s
[CV] END bootstrap=True, max_depth=50, max_features=1, min_samples_leaf=1, min_samples_split=10, n_estimators=1200; total time=  11.6s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1000; total time=  16.6s
[CV] END bootstrap=False, max_depth=60, max_features=1, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total tim

/Users/snesbitt/miniforge3/envs/py3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=False, max_depth=100, max_features=1, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  17.1s
[CV] END bootstrap=False, max_depth=100, max_features=1, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  18.1s
[CV] END bootstrap=False, max_depth=100, max_features=1, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  17.3s
[CV] END bootstrap=False, max_depth=100, max_features=1, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  17.6s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=  47.2s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=  46.5s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=  46.6s
[CV] END bootstrap=False, max_depth=80,

['finalized_model_rf.sav']